In [ ]:
import os

# Connecting and Mounting Blob Storage
storageAccountName = os.getenv("account_name")
storageAccountAccessKey =  os.getenv("account_key")
# sasToken = <sas-token>
blobContainerName = os.getenv("container_name")
mountPoint = "/mnt/data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
    #   extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

In [1]:
dbutils.fs.mounts()

In [2]:
dbutils.fs.ls("/mnt/data/glucose_dimensional_model")

In [ ]:
connectionString=os.getenv("EVENT_HUB_CONNECTION_STR")

FullConnectionString = f"{connectionString};EntityPath=increasing_trend_alert"

encrypted_conn_string = spark._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(FullConnectionString)

ehConfTrend = {
    "eventhubs.connectionString": encrypted_conn_string,
    "eventhubs.consumerGroup": "sparksql" 
}

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime, timedelta
import json


# Initialize Spark session
spark = SparkSession.builder.appName("GlucoseTrendAnalysis").getOrCreate()

# Loading the data from Azure Blob storage
df_glucose = spark.read.parquet("dbfs:/mnt/data/glucose_dimensional_model/fact_glucose_reading.parquet")
df_time = spark.read.parquet("dbfs:/mnt/data/glucose_dimensional_model/dim_time.parquet")

# Registering the DataFrames as temporary views so they can be queried  with SQL
df_glucose.createOrReplaceTempView("fact_glucose_reading")
df_time.createOrReplaceTempView("dim_time")

# As we scheduled the job on the 1st of every month, we use datetime in other to ensure that we are getting the data for the previous month

current_date = datetime.now()
first_day_of_current_month = current_date.replace(day=1)
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)
previous_month = last_day_of_previous_month.month
year = last_day_of_previous_month.year

# The SQL query to get the weekly average glucose levels for the previous month
sql_query = f"""
WITH WeeklyAverages AS (
  SELECT 
    user_id,
    YEAR(full_date) AS year,
    MONTH(full_date) AS month,
    CEIL(DAY(full_date) / 7.0) AS week_of_month,
    AVG(avg_glucose) AS avg_weekly_glucose
  FROM fact_glucose_reading
  JOIN dim_time ON fact_glucose_reading.date_key = dim_time.date_key
  WHERE YEAR(full_date) = {year} AND MONTH(full_date) = {previous_month}
  GROUP BY user_id, YEAR(full_date), MONTH(full_date), CEIL(DAY(full_date) / 7.0)
),
ConsecutiveIncrease AS (
  SELECT 
    user_id,
    year,
    month,
    week_of_month,
    avg_weekly_glucose,
    LAG(avg_weekly_glucose, 1) OVER (PARTITION BY user_id ORDER BY year, month, week_of_month) AS prev_week_glucose,
    CASE 
      WHEN avg_weekly_glucose > LAG(avg_weekly_glucose, 1) OVER (PARTITION BY user_id ORDER BY year, month, week_of_month) THEN 1
      ELSE 0
    END AS is_increasing
  FROM WeeklyAverages
),
UsersWithIncrease AS (
  SELECT 
    user_id
  FROM ConsecutiveIncrease
  WHERE month = {previous_month} AND year = {year}
  GROUP BY user_id
  HAVING SUM(is_increasing) = 4
)
SELECT 
  a.user_id,
  MAX(CASE WHEN a.week_of_month = 1 THEN a.avg_weekly_glucose ELSE NULL END) AS avg_week1,
  MAX(CASE WHEN a.week_of_month = 2 THEN a.avg_weekly_glucose ELSE NULL END) AS avg_week2,
  MAX(CASE WHEN a.week_of_month = 3 THEN a.avg_weekly_glucose ELSE NULL END) AS avg_week3,
  MAX(CASE WHEN a.week_of_month = 4 THEN a.avg_weekly_glucose ELSE NULL END) AS avg_week4,
  MAX(CASE WHEN a.week_of_month = 5 THEN a.avg_weekly_glucose ELSE NULL END) AS avg_week5
FROM WeeklyAverages a
JOIN UsersWithIncrease b ON a.user_id = b.user_id
GROUP BY a.user_id
"""

# Execute the SQL query
result_df = spark.sql(sql_query)

# Show the results
result_df.show()


+-------+------------------+------------------+------------------+------------------+------------------+
|user_id|         avg_week1|         avg_week2|         avg_week3|         avg_week4|         avg_week5|
+-------+------------------+------------------+------------------+------------------+------------------+
|    128|108.87064797537667|110.04942648751395|110.06106240408761|110.28838566371373|111.82491302490234|
|    134|109.31297302246094|109.60140773228237|109.98912702287946|110.04472351074219| 111.5343132019043|
+-------+------------------+------------------+------------------+------------------+------------------+



In [ ]:
# Checking if the DataFrame is non-empty
if result_df.count() > 0:
    increasing_trend_json_df = result_df.select(
        col("user_id").cast("string").alias("partitionKey"),
        to_json(struct(*[col(x) for x in result_df.columns])).alias("body")
    )

    # Preparing And Sending the data to Azure Event Hub
    increasing_trend_json_df\
        .write \
        .format("eventhubs") \
        .options(**ehConfTrend) \
        .save()
else:
    print(f"No users with 4 consecutive weeks of increasing glucose readings for the period {year}-{previous_month}")


In [ ]:
dbutils.fs.unmount('/mnt/data')